In [1]:
%pylab inline
import pandas as pd
from copy import deepcopy as copy

Populating the interactive namespace from numpy and matplotlib


#### todo: 
- set an upper limit on the time to use in the data series

Use data through: 30th June 2012?

Chrome v1.0–v12.0, Firefox v1.0–v5.0, IE v4.0–v9.0, and Safari v1.0–v5.0.6

### Collect the data

This analysis uses datasets supplied by the University of Trento in Italy: http://securitylab.disi.unitn.it/doku.php?id=datasets

The first step for us will be to format the datasets in a fashion that we can use for curve fitting. We begin by loading the datasets into a standard format, and merging them together.

In [2]:
iev = pd.read_excel('../data/iev.xlsx', sheetname='DATASET')
iev.columns = ['cve','bugname','cveDate','minVersion','maxVersion']
iev['product']='ie'
print iev.shape
iev.head(2)

(606, 6)


,cve,bugname,cveDate,minVersion,maxVersion,product
0,1999-0031,NaN,1997-07-08,4,4,ie
1,1999-0331,NaN,1998-01-01,4,4,ie


In [3]:
asv = pd.read_excel('../data/asv.xlsx', sheetname='DATASET')
asv.columns = ['cve','bugname','cveDate','minVersion','maxVersion']
asv['product']='safari'
print asv.shape
asv.head(2)

(552, 6)


,cve,bugname,cveDate,minVersion,maxVersion,product
0,2003-0355,NaN,2003-06-09,1,1,safari
1,2003-0370,NaN,2003-06-16,1,1,safari


In [4]:
ffv = pd.read_excel('../data/ffv.xlsx', sheetname='DATASET')
ffv.drop(['mfsa'], axis=1, inplace=True)
ffv.columns = ['bugname','cve','bugDate','cveDate','minVersion','maxVersion']
ffv['product']='ff'
print ffv.shape
ffv.head(2)

(2374, 7)


,bugname,cve,bugDate,cveDate,minVersion,maxVersion,product
0,273699,2004-1156,2004-12-08,2004-12-31,1.0,1.0,ff
1,NaN,2004-2657,NaT,2004-12-31,1.5,1.5,ff


In [5]:
gcv = pd.read_excel('../data/gcv.xlsx', sheetname='DATASET')
gcv.drop(['codeMinVer','codeMaxVer'], axis=1, inplace=True)
gcv.columns = ['bugname','cve','cveDate','bugDate','minVersion','maxVersion']
gcv['product']='chrome'
print gcv.shape
gcv.head(2)

(640, 7)


,bugname,cve,cveDate,bugDate,minVersion,maxVersion,product
0,3275,2010-0650,2010-02-18,2008-10-09,1,4,chrome
1,7338,2009-2060,2009-06-15,2009-02-03,1,1,chrome


In [6]:
vulns = pd.concat([gcv, ffv, asv, iev])
vulns.dropna(how='any',subset=['maxVersion', 'minVersion'], axis=0, inplace=True)

vuln_list = []
for i, row in vulns.iterrows(): #slowish, but whatever
    for version in range(int(row['minVersion']), int(row['maxVersion']+1)):
        row['version'] = version
        vuln_list.append(copy(row))
        
vulns = pd.DataFrame(vuln_list)
vulns.drop(['maxVersion', 'minVersion'], axis=1, inplace=True)

vulns.sort(columns=['product', 'version', 'cve'], inplace=True)
print vulns.shape
vulns.head(3)

(17714, 6)


/Users/houghton/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:13: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,bugDate,bugname,cve,cveDate,product,version
602,NaT,NaN,2008-5749,2008-12-29,chrome,1
603,NaT,NaN,2008-7294,2011-08-09,chrome,1
604,NaT,NaN,2009-0276,2009-02-03,chrome,1


### Break the dataset for each version into subsets:

Massacci and Nguyen's paper consider several ways of constituting the data. 

Excerpted from their paper:

> - NVD(r): a set of CVEs claiming r is vulnerable.
- NVD.Bug(r): a set of CVEs confirmed by at least a vendor bug report, and claiming r is vulnerable.
- NVD.Advice(r): a set of CVEs confirmed by at least a vendor advisory, and claiming r is vulnerable. Notice that the advisory report might not mention r, but later releases.
- NVD.Nbug(r): a set of vendor bug reports confirmed by a CVE claiming r is vulnerable.
- Advice.NBug(r): a set of bug reports mentioned in a vendor advisory report, which also refers to at least a CVE that claims r is vulnerable.

>Data set       | Definition
---------      | ----------
NVD(r)         | {nvd ∈ NVD &#124; r ∈ R_nvd} 
NVD.Bug(r)     | {nvd ∈ NVD &#124; ∃b ∈ BUG : r ∈ R_nvd ∧ idb ∈ refs_nvd}
NVD.Advice(r)  | {nvd ∈ NVD &#124; ∃a ∈ ADV : r ∈ R_nvd ∧ ida ∈ refs_nvd}
NVD.NBug(r)    | {b ∈ BUG &#124; ∃nvd ∈ NVD : r ∈ R_nvd ∧ idb ∈ refs_nvd}
Advice.NBug(r) | {b ∈ BUG &#124; ∃a ∈ ADV,∃nvd ∈ NVD : r ∈ R_nvd ∧idb ∈ refsa ∧ id_nvd ∈ refsa ∧ clustera(idb, id_nvd)}

>Note: Rnvd, refsnvd denote the vulnerable releases and references of an en- try nvd, respectively. ida, idb, idnvd denote the identifier of a, b, and nvd. clustera(idb, idnvd) is a predicate checking whether idb and idnvd are located next together in the advisory a.


Todo: in each of the below, throw out rows with less than 6 datapoints...

In [7]:
# corresponds to the NVD data set (to the best of my understanding)
NVD = vulns[['cveDate','cve','product','version']].drop_duplicates()
NVD.set_index(['product', 'version'], inplace=True)
NVD_counts = NVD.groupby(level=[0,1]).apply(lambda x: x.set_index('cveDate').resample('M', how='count').reset_index())
NVD_cumulative = NVD_counts['cve'].unstack().T.cumsum().T
NVD_cumulative.columns = NVD_cumulative.columns + 1
NVD_cumulative.insert(0, 0, 0, allow_duplicates=False)
print NVD_cumulative.shape
NVD_cumulative.head(2)

(56, 175)


0    1    2    3    4    5    6    7    8    9   ...   165  \
product version                                                   ...         
chrome  1          0    1    2    4    4    6    9   13   16   21 ...   NaN   
        2          0    1    2    3    6    9   12   12   16   16 ...   NaN   

                 166  167  168  169  170  171  172  173  174  
product version                                               
chrome  1        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
        2        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  

[2 rows x 175 columns]

In [8]:
NVD_cumulative.columns + 1

Int64Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
            ...
            166, 167, 168, 169, 170, 171, 172, 173, 174, 175],
           dtype='int64', length=175)

In [9]:
# corresponds to the union of the NVD.bug and NVD.advice datasets
NVD_Bug = vulns.dropna(subset=['bugname'])[['cveDate','cve','product','version']].drop_duplicates()
NVD_Bug.set_index(['product', 'version'], inplace=True)
NVD_Bug_counts = NVD_Bug.groupby(level=[0,1]).apply(lambda x: x.set_index('cveDate').resample('M', how='count').reset_index())
NVD_Bug_cumulative = NVD_Bug_counts['cve'].unstack().T.cumsum().T
NVD_Bug_cumulative.columns = NVD_Bug_cumulative.columns + 1
NVD_Bug_cumulative.insert(0, 0, 0, allow_duplicates=False)
print NVD_Bug_cumulative.shape
NVD_Bug_cumulative.head(2)

(56, 155)


0    1    2    3    4    5    6    7    8    9   ...   145  \
product version                                                   ...         
chrome  1          0    2    4    7    8   10   11   11   14   14 ...   NaN   
        2          0    1    2    3    4    6    8    8   11   11 ...   NaN   

                 146  147  148  149  150  151  152  153  154  
product version                                               
chrome  1        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
        2        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  

[2 rows x 155 columns]

In [10]:
# Corresponds to the NVD.Nbug dataset (to the best of my understanding)
Bug = vulns.dropna(subset=['bugname'])[['cveDate','cve','product','version']]
Bug.set_index(['product', 'version'], inplace=True)
Bug_counts = Bug.groupby(level=[0,1]).apply(lambda x: x.set_index('cveDate').resample('M', how='count').reset_index())
Bug_cumulative = Bug_counts['cve'].unstack().T.cumsum().T
Bug_cumulative.columns = Bug_cumulative.columns + 1
Bug_cumulative.insert(0, 0, 0, allow_duplicates=False)
Bug_cumulative.head(2)

0    1    2    3    4    5    6    7    8    9   ...   145  \
product version                                                   ...         
chrome  1          0    2    4    9   10   12   13   13   16   16 ...   NaN   
        2          0    1    2    3    4    6    8    8   11   11 ...   NaN   

                 146  147  148  149  150  151  152  153  154  
product version                                               
chrome  1        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
        2        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  

[2 rows x 155 columns]

In [11]:
# Corresponds to the Advice.Nbug dataset (to the best of my understanding)
Bug_date = vulns[['bugDate','cve','product','version']].dropna(subset=['bugDate'])
Bug_date.set_index(['product', 'version'], inplace=True)
Bug_date_counts = Bug_date.groupby(level=[0,1]).apply(lambda x: x.set_index('bugDate').resample('M', how='count').reset_index())
Bug_date_cumulative = Bug_date_counts['cve'].unstack().T.cumsum().T
Bug_date_cumulative.columns = Bug_date_cumulative.columns + 1
Bug_date_cumulative.insert(0, 0, 0, allow_duplicates=False)
Bug_date_cumulative.head(2)


0    1    2    3    4    5    6    7    8    9   ...   109  \
product version                                                   ...         
chrome  1          0    1    1    1    1    4    5   10   12   14 ...   NaN   
        2          0    1    1    1    1    2    3    6    8   10 ...   NaN   

                 110  111  112  113  114  115  116  117  118  
product version                                               
chrome  1        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
        2        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  

[2 rows x 119 columns]

In [14]:
datasets = pd.concat([NVD_cumulative, 
                     NVD_Bug_cumulative, 
                     Bug_cumulative, 
                     Bug_date_cumulative],
                    keys=['NVD',
                          'NVD.Bug', 
                          'NVD.Advice-Bug', 
                          'Advice.Nbug'])

obs_samples_list = []
for idx, row in datasets.iterrows(): #slowish, but whatever
    Tmax = row.notnull().sum()
    if Tmax >= 6: #need 6 months of data, plus month 0
        for t in range(6,Tmax):
            sample = row.loc[:t]
            sample['DS'] = idx[0] #there is probably a better way to do this
            sample['product'] = idx[1]
            sample['version'] = idx[2]
            sample['tseries'] = t
            obs_samples_list.append(sample)
        
obs_samples = pd.DataFrame(obs_samples_list)
obs_samples.set_index(['DS','product','version','tseries'], inplace=True)
print obs_samples.shape
obs_samples.head()

(5456, 175)


0    1    2    3    4    5    6    7    8    9    \
DS  product version tseries                                                     
NVD chrome  1       6          0    1    2    4    4    6    9  NaN  NaN  NaN   
                    7          0    1    2    4    4    6    9   13  NaN  NaN   
                    8          0    1    2    4    4    6    9   13   16  NaN   
                    9          0    1    2    4    4    6    9   13   16   21   
                    10         0    1    2    4    4    6    9   13   16   21   

                            ...   165  166  167  168  169  170  171  172  173  \
DS  product version tseries ...                                                 
NVD chrome  1       6       ...   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
                    7       ...   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
                    8       ...   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
                    9       ...   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
                    10      ...   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   

                             174  
DS  product version tseries       
NVD chrome  1       6        NaN  
                    7        NaN  
                    8        NaN  
                    9        NaN  
                    10       NaN  

[5 rows x 175 columns]

In [15]:
obs_samples.to_pickle('_!obs_samples!_.pickle')